In [4]:
import torch
import tqdm
import torchvision

from models import get_model

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = get_model("mobilenet", 2, True)
model.load_state_dict(torch.load("./rsc/outputs/saved_model/mobilenet.pth"))

<All keys matched successfully>

In [3]:
DATASET = "./../data/valid"

transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize(255),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=[.485,.456,.406], std=[.229,.224,.225])
    ])

dataset = torchvision.datasets.ImageFolder(root=DATASET, transform=transforms)
loader  = torch.utils.data.DataLoader(dataset, batch_size=4, pin_memory=True)

In [29]:
def test(model, loader, device='cpu'):

    model.to(device)
    model.eval()

    targets     = list()
    predictions = list()
    
    with torch.no_grad():
        for (sample, target) in tqdm.tqdm(loader):
            
            sample = sample.to(device)
            target = target.to(device)
            
            output = model(sample)
            
            _, predicted = torch.max(output, 1)
            
            predictions = predictions + predicted.cpu().numpy().tolist()
            targets     = targets     + target.cpu().numpy().tolist()

    return predictions, targets

preds, targets = test(model, loader, device=DEVICE)

100%|██████████| 25/25 [00:01<00:00, 22.39it/s]


In [30]:
print(preds)
print(targets)

[0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [33]:
from sklearn.metrics import accuracy_score, roc_auc_score

accuracy_score(targets, preds)

0.5

In [34]:
roc_auc_score(targets, preds)

0.5